In [9]:
from dotenv import load_dotenv
import itertools
from openai import OpenAI
from pprint import pprint
from datasets import load_dataset
import json

In [2]:

client = OpenAI()

### revise openai completion

In [3]:


response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)

In [4]:
pprint(json.loads(response.json()))

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'content': 'The 2020 World Series was played at '
                                     'Globe Life Field in Arlington, Texas.',
                          'function_call': None,
                          'role': 'assistant',
                          'tool_calls': None}}],
 'created': 1703104816,
 'id': 'chatcmpl-8XxeSupkHUzWmCyYEZ0378XOMzRiK',
 'model': 'gpt-3.5-turbo-0613',
 'object': 'chat.completion',
 'system_fingerprint': None,
 'usage': {'completion_tokens': 17, 'prompt_tokens': 53, 'total_tokens': 70}}


In [5]:
response_json = json.loads(response.json())

In [6]:
response_json['choices'][0]['message']['content']

'The 2020 World Series was played at Globe Life Field in Arlington, Texas.'

### :Load_dataset

In [18]:
#pretrained_dataset = load_dataset("EleutherAI/pile", split="train", streaming=True)
#use to train 
dataset = load_dataset("c4", "en", split="train", streaming=True)

In [19]:
m = 10
top_n = tuple(itertools.islice(dataset, m))



In [20]:
pprint(top_n[0])

{'text': 'Beginners BBQ Class Taking Place in Missoula!\n'
         'Do you want to get better at making delicious BBQ? You will have the '
         'opportunity, put this on your calendar now. Thursday, September 22nd '
         'join World Class BBQ Champion, Tony Balay from Lonestar Smoke '
         'Rangers. He will be teaching a beginner level class for everyone who '
         'wants to get better with their culinary skills.\n'
         'He will teach you everything you need to know to compete in a KCBS '
         'BBQ competition, including techniques, recipes, timelines, meat '
         'selection and trimming, plus smoker and fire information.\n'
         'The cost to be in the class is $35 per person, and for spectators it '
         'is free. Included in the cost will be either a t-shirt or apron and '
         'you will be tasting samples of each meat that is prepared.',
 'timestamp': '2019-04-25T12:57:54Z',
 'url': 'https://klyq.com/beginners-bbq-class-taking-place-in-misso

In [21]:
text_list = tuple(map(lambda x: x['text'], top_n))
text_list[1]

'Discussion in \'Mac OS X Lion (10.7)\' started by axboi87, Jan 20, 2012.\nI\'ve got a 500gb internal drive and a 240gb SSD.\nWhen trying to restore using disk utility i\'m given the error "Not enough space on disk ____ to restore"\nBut I shouldn\'t have to do that!!!\nAny ideas or workarounds before resorting to the above?\nUse Carbon Copy Cloner to copy one drive to the other. I\'ve done this several times going from larger HDD to smaller SSD and I wound up with a bootable SSD drive. One step you have to remember not to skip is to use Disk Utility to partition the SSD as GUID partition scheme HFS+ before doing the clone. If it came Apple Partition Scheme, even if you let CCC do the clone, the resulting drive won\'t be bootable. CCC usually works in "file mode" and it can easily copy a larger drive (that\'s mostly empty) onto a smaller drive. If you tell CCC to clone a drive you did NOT boot from, it can work in block copy mode where the destination drive must be the same size or larg

### prompt template

In [29]:
template = """ 
Given the text below, generate two question-and-answer pairs from the text in jsonl format, where the keys are "question and "answer"
"{text}"

"""

In [30]:

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": template.format(text = text_list[0])},
  ]
)

In [32]:
pprint(json.loads(response.json())["choices"][0]["message"]["content"])

('[\n'
 '  {"question": "Who is teaching the beginners BBQ class?", "answer": "Tony '
 'Balay from Lonestar Smoke Rangers."},\n'
 '  {"question": "What will be included in the cost of the class?", "answer": '
 '"A t-shirt or apron and tasting samples of each prepared meat."}\n'
 ']')


In [49]:
def get_response(text: str):
    template = """ 
    Given the text below, generate two question-and-answer pairs from the text as a list of json, where the keys are "question and "answer"
    do not try to make up questions that are not in the text, you can skip questions if you need to.
    "{text}"

    """
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": template.format(text=text)},
        ],
    )
    response_msg = json.loads(response.json())["choices"][0]["message"]["content"]
    # return json.loads(response_msg)

    try:
        result = json.loads(response_msg)
    except:
        print(response_msg)
    return result

In [50]:
import time

result_list = []
for text in text_list[:5]:
    result = get_response(text)
    # pprint(result)
    result_list.extend(result)
    time.sleep(10)

In [51]:
pprint(result_list)

[{'answer': 'Tony Balay from Lonestar Smoke Rangers.',
  'question': 'Who will be teaching the BBQ class in Missoula?'},
 {'answer': '$35 per person.',
  'question': 'How much does it cost to participate in the class?'},
 {'answer': "The person is receiving the error message 'Not enough space on "
            "disk ____ to restore.'",
  'question': 'What error message is the person receiving when trying to '
              'restore using disk utility?'},
 {'answer': 'The person suggests using Carbon Copy Cloner to copy one drive to '
            'the other.',
  'question': 'What method does the person suggest using to copy one drive to '
              'the other?'},
 {'answer': 'The shortall is made of foil plaid lycra and spandex with '
            'metallic slinky insets.',
  'question': 'What material is the shortall made of?'},
 {'answer': 'The shortall comes with an attached metallic elastic belt with an '
            'O-ring and a headband.',
  'question': 'What is included with t

In [53]:
with jsonlines.open(f'json/gpt_c4.jsonl', 'w') as writer:
    writer.write_all(result_list)